In [1]:
# This code is developed by Pooyan :D

# Assignment Tittle
__Author__ : Pooyan oskouhi 400222003
__Course__ : Undergraduate Numerical Analysis Course

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sympy as sp

# Problem 1
Assume $ f(x) = arctan(x) $.Write cubic spline with 11 points in terval [0, 1].

In [2]:
x = sp.symbols('x')
f = sp.sqrt(x) * sp.atan(x)

In [3]:
x_values = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
y_values = [float(f.subs(x, i)) for i in x_values]
y_values

[0.0,
 0.031517995319188724,
 0.08827797805619232,
 0.15963746087367492,
 0.24065336317880726,
 0.32784836840539894,
 0.4186071449027414,
 0.5109700015709241,
 0.6035066456056488,
 0.695209447624046,
 0.7853981633974483]